@Abdulrahman Alabrash 

https://github.com/alabrashJr/DCNN-Julia


In [1]:
using Pkg; for p in ("HDF5","JLD2","FileIO"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using HDF5,JLD2,FileIO

┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.1/HDF5/L7Dga.ji for HDF5 [f67ccb44-e63f-5c2f-98bd-6dc0ccc4ba2f]
└ @ Base loading.jl:1184


In [2]:
mutable struct node 
    word
    kidsword
    kidsindex
    parent
    finished
    is_word
    selfindex
    parentindex
    label
    ind
node(word) = word == nothing ? new(nothing,nothing,nothing,nothing,nothing,0,nothing,nothing,nothing,nothing) : new(word,[],[],[],0,1,0,0,"",-1)
    
end

In [3]:
input_file  = "DCNN/TREC/TREC_all_parsed.txt";
output_file = "DCCN/TREC/TREC_all_tree.p";

In [4]:
input = open(input_file,"r")
counter =0
doc = []
    for i in readlines(input)
        # i
        counter += 1
         #counter
        #i = strip(i)
        i = strip(i,['[',']',' '])
        #print i
        current = split(i,"), ")
        node_container = Dict()
        ROOT = node("ROOT")
        node_container[0] = ROOT    
        for (index,j) in enumerate(current)
            label = split(j,'(')[1]
            j = strip((split(j,'(')[2]),')')
            current_list = [strip(i) for i in split(j,", ")]
            current_node=node(join(split(current_list[2],'-')[1:end-1],'_'))
            current_node.selfindex=parse(Int,split(current_list[2],'-')[end])
            current_node.label = label
            
        if join(split(current_list[1],'-')[1:end-1],'_') == "ROOT"
                current_node.parent = ROOT
            else               
            current_node.parent=join(split(current_list[1],'-')[1:end-1],'_')
            current_node.parentindex=parse(Int,split(current_list[1],'-')[end])
             node_container[current_node.selfindex] = current_node
            end
        end
        node_container1 = deepcopy(node_container)
        for (index,current) in node_container1
            p_index = current.parentindex
            #print p_index
            if p_index ∉ keys(node_container)
                new_node = node(current.parent)
                new_node.parent = ROOT
                new_node.selfindex = p_index
                node_container[p_index] = new_node    
                end
        end
        for (index,current) in node_container1
            if current.word == "ROOT"
                continue
            end
            p_index = current.parentindex
            #print p_index
            if p_index ∉ keys(node_container)
                new_node = node(current.parent)
                new_node.parent = ROOT
                new_node.selfindex = p_index
                node_container[p_index] = new_node
            end
            p = node_container[p_index]
            
            push!(p.kidsword,current.word)
            push!(p.kidsindex,current.selfindex)
        end
        for (j,i) in node_container
            println( "self\t", i.selfindex, "\tparent\t", i.parentindex,"\tchild\t", i.kidsindex)
        end
        push!(doc,node_container)
    end
     #length(doc)
    

SystemError: SystemError: opening file "DCNN/TREC/TREC_all_parsed.txt": No such file or directory

In [5]:
@doc save

  * `save(filename, data...)` saves the contents of a formatted file,

trying to infer the format from `filename`.

  * `save(Stream(format"PNG",io), data...)` specifies the format directly, and bypasses inference.
  * `save(File(format"PNG",filename), data...)` specifies the format directly, and bypasses inference.
  * `save(f, data...; options...)` passes keyword arguments on to the saver.


In [6]:
save("data.jld2","data",doc) # using FileIO pacakge 

UndefVarError: UndefVarError: doc not defined

In [7]:
#to do change JLD to JLD2 https://github.com/JuliaIO/JLD2.jl